# Dataset Construction
save the masks and images as .png in the following folder structure:

```
dataset/   #Primary data folder for the project
├── input/           #All input data is stored here. 
│   ├── train_images/
│   │   ├── image01.png
│   │   ├── image02.png
│   │   └── ...
│   ├── train_masks/        #All binary masks organized in respective sub-directories.
│   │   ├── class1/
│   │   │   ├── image01.png
│   │   │   ├── image02.png
│   │   │   └── ...
│   │   ├── class2/
│   │   │   ├── image01.png
│   │   │   ├── image02.png
│   │   │   └── ...
│   ├── val_images/         #Validation images
│   │   ├── image01.png
│   │   ├── image02.png
│   │   └── ...
│   ├── val_masks/          #Validation masks organized in respective sub-directories.
│   │   ├── class1/
│   │   │   ├── image01.png
│   │   │   ├── image02.png
│   │   │   └── ...
│   │   ├── class2/
│   │   │   ├── image01.png
│   │   │   ├── image02.png
│   │   │   └── ...
```

In [7]:
import glob
import json
import os
import cv2

train_mask_path = os.path.join("dataset", "input", "train_masks")
train_json_path = os.path.join("dataset", "input", "train_images", "train.json")
val_mask_path = os.path.join("dataset", "input", "val_masks")
val_json_path = os.path.join("dataset", "input", "val_images", "val.json")
train_mask_path = os.path.join(os.getcwd(), train_mask_path)
train_json_path = os.path.join(os.getcwd(), train_json_path)
val_mask_path = os.path.join(os.getcwd(), val_mask_path)
val_json_path = os.path.join(os.getcwd(), val_json_path)


# Label IDs of the dataset representing different categories
category_ids = {
    "class1": 1, #make sure the name of the class is the same as the folder name in the dataset 
}

MASK_EXT = 'png'
ORIGINAL_EXT = 'png'
image_id = 0
annotation_id = 0

In [ ]:

def images_annotations_info(maskpath):
    """
    Process the binary masks and generate images and annotations information.

    :param maskpath: Path to the directory containing binary masks
    :return: Tuple containing images info, annotations info, and annotation count
    """
    global image_id, annotation_id
    annotations = []
    images = []

    # Iterate through categories and corresponding masks
    for category in category_ids.keys():
        for mask_image in glob.glob(os.path.join(maskpath, category, f'*.{MASK_EXT}')):
            original_file_name = f'{os.path.basename(mask_image).split(".")[0]}.{ORIGINAL_EXT}'
            mask_image_open = cv2.imread(mask_image)
            
            # Get image dimensions
            height, width, _ = mask_image_open.shape

            # Create or find existing image annotation
            if original_file_name not in map(lambda img: img['file_name'], images):
                image = {
                    "id": image_id + 1,
                    "width": width,
                    "height": height,
                    "file_name": original_file_name,
                }
                images.append(image)
                image_id += 1
            else:
                image = [element for element in images if element['file_name'] == original_file_name][0]

            # Find contours in the mask image
            gray = cv2.cvtColor(mask_image_open, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]

            # Create annotation for each contour
            for contour in contours:
                bbox = cv2.boundingRect(contour)
                area = cv2.contourArea(contour)
                segmentation = contour.flatten().tolist()

                annotation = {
                    "iscrowd": 0,
                    "id": annotation_id,
                    "image_id": image['id'],
                    "category_id": category_ids[category],
                    "bbox": bbox,
                    "area": area,
                    "segmentation": [segmentation],
                }

                # Add annotation if area is greater than zero
                if area > 0:
                    annotations.append(annotation)
                    annotation_id += 1

    return images, annotations, annotation_id


def process_masks(mask_path, dest_json):
    global image_id, annotation_id
    image_id = 0
    annotation_id = 0

    # Initialize the COCO JSON format with categories
    coco_format = {
        "info": {},
        "licenses": [],
        "images": [],
        "categories": [{"id": value, "name": key, "supercategory": key} for key, value in category_ids.items()],
        "annotations": [],
    }

    # Create images and annotations sections
    coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

    # Save the COCO JSON to a file
    print("Saving annotations to file: %s" % dest_json)
    with open(dest_json, "w") as outfile:
        json.dump(coco_format, outfile, sort_keys=True, indent=4)

    print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

process_masks(train_mask_path, train_json_path)
process_masks(val_mask_path, val_json_path)

# test_mask_path = ""
# test_json_path = ""
# process_masks(test_mask_path, test_json_path)

    


In [9]:
base_input_path = os.path.join("dataset", "input")
base_input_path = os.path.join(os.getcwd(), base_input_path)

base_output_path = os.path.join("dataset", "yolo_dataset")
base_output_path = os.path.join(os.getcwd(), base_output_path)

#where the yaml file will be saved
train_path = os.path.join("dataset", "yolo_dataset", "train", "images")
train_path = os.path.join(os.getcwd(), train_path)
val_path = os.path.join("dataset", "yolo_dataset", "valid", "images")
val_path = os.path.join(os.getcwd(), val_path)

In [10]:
import json
import os
import shutil
import yaml

# Function to convert images to YOLO format
def convert_to_yolo(input_images_path, input_json_path, output_images_path, output_labels_path):
    # Open JSON file containing image annotations
    f = open(input_json_path)
    data = json.load(f)
    f.close()

    # Create directories for output images and labels
    os.makedirs(output_images_path, exist_ok=True)
    os.makedirs(output_labels_path, exist_ok=True)

    # List to store filenames
    file_names = []
    for filename in os.listdir(input_images_path):
        if filename.endswith(".png"):
            source = os.path.join(input_images_path, filename)
            destination = os.path.join(output_images_path, filename)
            shutil.copy(source, destination)
            file_names.append(filename)

    # Function to get image annotations
    def get_img_ann(image_id):
        return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

    # Function to get image data
    def get_img(filename):
        return next((img for img in data['images'] if img['file_name'] == filename), None)

    # Iterate through filenames and process each image
    for filename in file_names:
        img = get_img(filename)
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']
        img_ann = get_img_ann(img_id)

        # Write normalized polygon data to a text file
        if img_ann:
            with open(os.path.join(output_labels_path, f"{os.path.splitext(filename)[0]}.txt"), "a") as file_object:
                for ann in img_ann:
                    current_category = ann['category_id'] - 1
                    polygon = ann['segmentation'][0]
                    normalized_polygon = [format(coord / img_w if i % 2 == 0 else coord / img_h, '.6f') for i, coord in enumerate(polygon)]
                    file_object.write(f"{current_category} " + " ".join(normalized_polygon) + "\n")

# Function to create a YAML file for the dataset
def create_yaml(input_json_path, output_yaml_path, train_path, val_path, test_path=None):
    with open(input_json_path) as f:
        data = json.load(f)
    
    # Extract the category names
    names = [category['name'] for category in data['categories']]
    
    # Number of classes
    nc = len(names)

    # Create a dictionary with the required content
    yaml_data = {
        'names': names,
        'nc': nc,
        'test': test_path if test_path else '',
        'train': train_path,
        'val': val_path
    }

    # Write the dictionary to a YAML file
    with open(output_yaml_path, 'w') as file:
        yaml.dump(yaml_data, file, default_flow_style=False)


# Processing validation dataset (if needed)
convert_to_yolo(
    input_images_path=os.path.join(base_input_path, "val_images"),
    input_json_path=os.path.join(base_input_path, "val_images/val.json"),
    output_images_path=os.path.join(base_output_path, "valid/images"),
    output_labels_path=os.path.join(base_output_path, "valid/labels")
)

# Processing training dataset 
convert_to_yolo(
    input_images_path=os.path.join(base_input_path, "train_images"),
    input_json_path=os.path.join(base_input_path, "train_images/train.json"),
    output_images_path=os.path.join(base_output_path, "train/images"),
    output_labels_path=os.path.join(base_output_path, "train/labels")
)

# Creating the YAML configuration file
create_yaml(
    input_json_path=os.path.join(base_input_path, "train_images/train.json"),
    output_yaml_path=os.path.join(base_output_path, "data.yaml"),
    train_path=train_path,
    val_path=val_path,
    # test_path='../test/images'  # or None if not applicable
    test_path=None  # or None if not applicable
)
